In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [7]:
def calc_mutual_inductance(coil_1, coil_2, d=0, po=0, fi=0):
    """Расчёт взаимной индуктивности между двумя витками.
    :param coil_1: np.ndarray с радиусом одной катушки индуктивности, м
    :type coil_1: np.ndarray
    
    :param coil_2: список с радиусом одной катушки индуктивности, м
    :type coil_1: np.ndarray
    
    :param d: расстояние между витками, м
    :type d: np.ndarray or float, int
    
    :param po: латеральное смещение между витками, м
    :type po: np.ndarray or float, int
    
    :param fi: угловое смещение между витками
    :type fi: np.ndarray or float, int
    
    :rtype: np.ndarray
    :return: M, собственная индуктивность между витками
    """
    coil_1 = coil_1.T
    mu0 = 1.26 * 1e-6
    M = 0
    N = 360
    K = 360
    df1 = 2 * np.pi / N
    df2 = 2 * np.pi / K
    for n in range(N):
        for k in range(K):
            xk_xn = po + coil_1 * np.cos(df2 * k) * np.cos(fi) - coil_2 * np.cos(df1 * n)
            yk_yn = coil_1 * np.sin(df2 * k) * np.cos(fi) - coil_2 * np.sin(df1 * n)
            zk_zn = d + coil_1 * np.cos(df2 * k) * np.sin(fi)
            r12 = (xk_xn ** 2 + yk_yn ** 2 + zk_zn ** 2) ** 0.5
            M += (np.cos(df2 * k - df1 * n) * df1 * df2) / r12
    M *= mu0 * coil_1 * coil_2 / (4 * np.pi)
    M = M[np.isfinite(M)]
    return M

def calc_self_inductance(coil, thin=0.0004):
    mu0 = 1.26 * 1e-6
    L = np.sum(mu0 * coil * (np.log(8 * coil / thin) - 2))
    L += np.sum(calc_mutual_inductance(coil, coil))
    return L

Reproduction the result from article: Self-inductance of Multi-turn Round Inductive Coils with Production-specific Geometric Features

In [8]:
case1 = np.linspace(0.0225, 0.0475, 6, endpoint=True).reshape(1,6)
case2 = np.linspace(0.024, 0.048, 7, endpoint=True).reshape(1,7)
case3 = np.linspace(0.024, 0.048, 4, endpoint=True).reshape(1,4)
case4 = np.linspace(0.02, 0.044, 3, endpoint=True).reshape(1,3)
case5 = np.linspace(0.02, 0.06, 11, endpoint=True).reshape(1,11)
case6 = np.linspace(0.0035, 0.02, 12, endpoint=True).reshape(1,12)
case7 = np.linspace(0.005, 0.02, 7, endpoint=True).reshape(1,7)
print("case1:", calc_self_inductance(case1)*10**6, "мкГн")
print("case2:", calc_self_inductance(case2)*10**6, "мкГн")
print("case3:", calc_self_inductance(case3)*10**6, "мкГн")
print("case4:", calc_self_inductance(case4)*10**6, "мкГн")
print("case5:", calc_self_inductance(case5)*10**6, "мкГн")
print("case6:", calc_self_inductance(case6, thin=0.00025)*10**6, "мкГн")
print("case7:", calc_self_inductance(case7, thin=0.00025)*10**6, "мкГн")

/tmp/ipykernel_11560/984826311.py:34: RuntimeWarning: divide by zero encountered in true_divide
  M += (np.cos(df2 * k - df1 * n) * df1 * df2) / r12


case1: 3.031677273873793 мкГн
case2: 4.328442283273869 мкГн
case3: 1.5002662151289035 мкГн
case4: 0.7644171319554103 мкГн
case5: 9.778300272309695 мкГн
case6: 2.7426608935695787 мкГн
case7: 1.1057617515854272 мкГн
